# Open3D Guide: 2. Point Clouds

Source: [https://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html](https://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html).

Summary of contents:

- Visualize a point cloud: `o3d.visualization.draw_geometries()`
- Voxel downsampling: `pc.voxel_down_sample()`
- Vertex normal estimation: `pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(...)`
- Access estimated vertex normals as Numpy arrays: `np.asarray(pc.normals)[:10, :]`
- Crop point cloud: `cropped_pc = polygon_volume.crop_point_cloud(pc)`
- Paint point cloud: `pc.paint_uniform_color([1, 0.5, 0])`
- Point cloud distance and selection: `dists = pc1.compute_point_cloud_distance(pc2)`, `pc.select_by_index(ind)`
- Bounding volumes (AABB, OBB): `pc.get_axis_aligned_bounding_box()`, `pc.get_oriented_bounding_box()`
- Convex hull and sampling: `pc = mesh.sample_points_poisson_disk()`, `hull_mes, _ = pc.compute_convex_hull()`
- DBSCAN clustering: `pc.cluster_dbscan(eps=0.02, min_points=10, print_progress=True)`
- Plane segmentation: `plane_model, inliers = pc.segment_plane()`
- (Visually) Hidden point removal: `pc.hidden_point_removal(camera, radius)`

In [2]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Visualize point cloud

In [4]:
print("Load a ply point cloud, print it, and render it")
pcd = o3d.io.read_point_cloud("../models/fragment.ply")
print(pcd)
print(np.asarray(pcd.points))
# A new visualization window is opened
# Points rendered as surfels
# Keys:
#  [/]          : Increase/decrease field of view.
#  R            : Reset view point.
#  Ctrl/Cmd + C : Copy current view status into the clipboard.
#  Ctrl/Cmd + V : Paste view status from clipboard.
#  Q, Esc       : Exit window.
#  H            : Print help message.
#  P, PrtScn    : Take a screen capture.
#  D            : Take a depth capture.
#  O            : Take a capture of current rendering settings.
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Load a ply point cloud, print it, and render it
PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]
[Open3D INFO]   -- Mouse view control --
[Open3D INFO]     Left button + drag         : Rotate.
[Open3D INFO]     Ctrl + left button + drag  : Translate.
[Open3D INFO]     Wheel button + drag        : Translate.
[Open3D INFO]     Shift + left button + drag : Roll.
[Open3D INFO]     Wheel                      : Zoom in/out.
[Open3D INFO] 
[Open3D INFO]   -- Keyboard view control --
[Open3D INFO]     [/]          : Increase/decrease field of view.
[Open3D INFO]     R            : Reset view point.
[Open3D INFO]     Ctrl/Cmd + C : Copy current view status into the clipboard.
[Open3D INFO]     Ctrl/Cmd + V : Paste view status from clipboard.
[Open3D INFO] 
[Open3D INFO]   -- General control --


## Voxel downsampling

In [5]:
# Voxel downsampling
# 1. Points are bucketed into voxels.
# 2. Each occupied voxel generates exactly one point by averaging all points inside.
print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Downsample the point cloud with a voxel of 0.05


## Vertex normal estimation

In [6]:
print("Recompute the normal of the downsampled point cloud")
# Compute normals: estimate_normals()
# The function finds adjacent points and calculates the principal axis of the adjacent points using covariance analysis.
# The function takes an instance of KDTreeSearchParamHybrid class as an argument. 
# The two key arguments radius = 0.1 and max_nn = 30 specifies search radius and maximum nearest neighbor.
# It has 10cm of search radius, and only considers up to 30 neighbors to save computation time.
# NOTE: normal direction is chosen to comply with original ones, else arbitrary
downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)
# Visualize points and normals: toggle on/off normals with N
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True)

Recompute the normal of the downsampled point cloud


## Access estimated vertex normals as Numpy arrays

In [9]:
print("Print a normal vector of the 0th point")
print(downpcd.normals[0])

Print a normal vector of the 0th point
[ 0.85641574  0.01693013 -0.51600915]


In [10]:
# Use help() extensively to check all available variables/proterties/functions!
help(downpcd)

Help on PointCloud in module open3d.cpu.pybind.geometry object:

class PointCloud(Geometry3D)
 |  PointCloud class. A point cloud consists of point coordinates, and optionally point colors and point normals.
 |  
 |  Method resolution order:
 |      PointCloud
 |      Geometry3D
 |      Geometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(...)
 |      __add__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __copy__(...)
 |      __copy__(self: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __deepcopy__(...)
 |      __deepcopy__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: dict) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __iadd__(...)
 |      __iadd__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.Point

In [8]:
# Normal vectors can be transformed as a numpy array using np.asarray
print("Print the normal vectors of the first 10 points")
print(np.asarray(downpcd.normals)[:10, :])

Print the normal vectors of the first 10 points
[[ 8.56415744e-01  1.69301342e-02 -5.16009150e-01]
 [-3.10071169e-01  3.92564590e-02 -9.49902522e-01]
 [-2.21066308e-01  2.07235365e-07 -9.75258780e-01]
 [-2.65577574e-01 -1.84601949e-01 -9.46250851e-01]
 [-7.91944115e-01 -2.92017206e-02 -6.09894891e-01]
 [-8.84912237e-02 -9.89400811e-01  1.15131831e-01]
 [ 6.28492508e-01 -6.12988948e-01 -4.78791935e-01]
 [ 7.28260110e-01 -4.73518839e-01 -4.95395924e-01]
 [-5.07368635e-03 -9.99572767e-01 -2.87844085e-02]
 [ 3.49295119e-01  1.16948013e-02 -9.36939780e-01]]


## Crop point cloud

In [13]:
# Download the cropping demo
# The demo consists of the living room PLY `fragment.ply` and a JSON which contains a bounding polygon
demo_crop = o3d.data.DemoCropPointCloud()

The demo consists of the living room PLY `fragment.ply` and a JSON which contains a bounding polygon:
 
```json
{
	"axis_max" : 4.022921085357666,
	"axis_min" : -0.76341366767883301,
	"bounding_polygon" : 
	[
		[ 2.6509309513852526, 0.0, 1.6834473132326844 ],
		[ 2.5786428246917148, 0.0, 1.6892074266735244 ],
		[ 2.4625790337552154, 0.0, 1.6665777078297999 ],
		[ 2.2228544982251655, 0.0, 1.6168160446813649 ],
		[ 2.166993206001413, 0.0, 1.6115495157201662 ],
		[ 2.1167895865303286, 0.0, 1.6257706054969348 ],
		[ 2.0634657721747383, 0.0, 1.623021658624539 ],
		[ 2.0568612343437236, 0.0, 1.5853892911207643 ],
		[ 2.1605399001237027, 0.0, 0.96228993255083017 ],
		[ 2.1956669387205228, 0.0, 0.95572746049785073 ],
		[ 2.2191318790575583, 0.0, 0.88734449982108754 ],
		[ 2.2484881847925919, 0.0, 0.87042807267013633 ],
		[ 2.6891234157295827, 0.0, 0.94140677988967603 ],
		[ 2.7328692490470647, 0.0, 0.98775740674840251 ],
		[ 2.7129337547575547, 0.0, 1.0398850034649203 ],
		[ 2.7592174072415405, 0.0, 1.0692940558509485 ],
		[ 2.7689216419453428, 0.0, 1.0953914441371593 ],
		[ 2.6851455625455669, 0.0, 1.6307334122162018 ],
		[ 2.6714776099981239, 0.0, 1.675524657088997 ],
		[ 2.6579576128816544, 0.0, 1.6819127849749496 ]
	],
	"class_name" : "SelectionPolygonVolume",
	"orthogonal_axis" : "Y",
	"version_major" : 1,
	"version_minor" : 0
}
```

In [14]:
# Once we have the polygon which encloses our desired region, cropping is easy
print("Load a polygon volume and use it to crop the original point cloud")
# Read a json file that specifies polygon selection area
vol = o3d.visualization.read_selection_polygon_volume(
    "../models/cropped.json"
)
# Filter out points. Only the chair remains.
chair = vol.crop_point_cloud(pcd)
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

Load a polygon volume and use it to crop the original point cloud


## Paint point cloud

In [15]:
print("Paint chair")
# Paint all the points to a uniform color.
# The color is in RGB space, [0, 1] range.
chair.paint_uniform_color([1, 0.706, 0])
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

Paint chair


## Point Cloud Distance and Selection

In [16]:
# Load data
pcd = o3d.io.read_point_cloud("../models/fragment.ply")
vol = o3d.visualization.read_selection_polygon_volume(
    "../models/cropped.json")
chair = vol.crop_point_cloud(pcd)

# Compute the distance from a source point cloud to a target point cloud.
# I.e., it computes for each point in the source point cloud the distance to the closest point in the target point cloud
# pcd: 196133 points
# chair: 31337 points
# dists: 196133 items
# np.where yields a tuple witha unique array -> [0]
# With select_by_index all indices from pcd are taken which have a distance larger than 0.01
# Since chair is contained in pcd, this is equivalent to removing chair from pcd
dists = pcd.compute_point_cloud_distance(chair)
dists = np.asarray(dists)
ind = np.where(dists > 0.01)[0]
pcd_without_chair = pcd.select_by_index(ind)
o3d.visualization.draw_geometries([pcd_without_chair],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

## Bounding Volumes: AABB, OBB

In [23]:
# Get the AABB and the OBB of a point cloud
# Then visualize them
aabb = chair.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
obb = chair.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([chair, aabb, obb],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

## Convex Hull and Sampling

In [24]:
# Download data
bunny = o3d.data.BunnyMesh()
bunny_mesh = o3d.io.read_triangle_mesh(bunny.path) # ../models/BunnyMesh.ply

In [28]:
# Before computing the convex hull, the point cloud is sampled.
# sample_points_poisson_disk(): each point has approximately the same distance
# to the neighbouring points (blue noise).
# Method is based on Yuksel, "Sample Elimination for Generating Poisson Disk Sample Sets", EUROGRAPHICS, 2015
# number_of_points: Number of points that should be sampled.
pcl = bunny_mesh.sample_points_poisson_disk(number_of_points=2000)

# Compute the convex hull of the sampled point cloud (based in Qhull)
# A triangle mesh is returned
hull, _ = pcl.compute_convex_hull()
# The conv hull traingle mesh a line set is created for visualization purposes
# and lines painted in red
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))

# Visualize downsampled pointcloud as well as convex hull represented with lines
o3d.visualization.draw_geometries([pcl, hull_ls])

## DBSCAN Clustering

In [29]:
import matplotlib.pyplot as plt

In [30]:
# Load model
pcd = o3d.io.read_point_cloud("../models/fragment.ply")

# DBSCAN two parameters:
# - eps defines the distance to neighbors in a cluster 
# - and min_points defines the minimum number of points required to form a cluster.
# The function returns labels, where the label -1 indicates noise.
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True)
    )

# Plot points with colors
max_label = labels.max()
print(f"Point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.455,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 10
point cloud has 10 clusters


## Plane segmentation

In [32]:
# Segmententation of geometric primitives (only plane?) from point clouds using RANSAC
# - distance_threshold defines the maximum distance a point can have to an estimated plane to be considered an inlier,
# - ransac_n defines the number of points that are randomly sampled to estimate a plane,
# - and num_iterations defines how often a random plane is sampled and verified.
# The function then returns the plane as (a,b,c,d) such that for each point (x,y,z) on the plane we have ax+by+cz+d=0.
# The function further returns a list of indices of the inlier points.
pcd = o3d.io.read_point_cloud("../models/fragment.pcd")
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
# Plane model
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# Plot
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                  zoom=0.8,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

Plane equation: -0.05x + -0.10y + 0.99z + -1.06 = 0


## Hidden point removal

In [33]:
# Download data
armadillo = o3d.data.ArmadilloMesh()
armadillo_mesh = o3d.io.read_triangle_mesh(armadillo.path) # ../models/ArmadilloMesh.ply

In [34]:
# First, we load a mesh and sample points on it
print("Convert mesh to a point cloud and estimate dimensions")
pcd = armadillo_mesh.sample_points_poisson_disk(5000)
diameter = np.linalg.norm(
    np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound())
)
o3d.visualization.draw_geometries([pcd])

Convert mesh to a point cloud and estimate dimensions


In [36]:
# Imagine you want to render a point cloud from a given view point, 
# but points from the background leak into the foreground because they are not occluded by other points.
# For this purpose we can apply a hidden point removal algorithm
print("Define parameters used for hidden_point_removal")
camera = [0, 0, diameter]
radius = diameter * 100

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd = pcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([pcd])

Define parameters used for hidden_point_removal
Get all points that are visible from given view point
Visualize result
